In [1]:
# Import das bibliotecas necessárias

import pandas as pd
import numpy as np
import pyautogui as gui
from datetime import datetime
from tkinter import *
from tkinter import ttk
from tkinter.filedialog import askopenfilename

In [2]:
'''
Função que realiza as seguintes ações:

- Importa as tabelas de pedido e cotação necessárias;
- Faz o tratamento dos dados para que a saída seja do jeito desejado;
- Digita automaticamente as quantidades, produtos e preços no ERP.

'''

def AutPed(e_text0, e_text1, e_text2, e_text3, e_text4, e_text5, e_text6, e_text7):
    cliente = e_text0

    if cliente == 'Rappi':

        pedido = e_text1
        sheet = e_text3
        df = pd.read_excel(pedido)
        oc = df[list(df.columns)[1]][int(sheet) + 3][df[list(df.columns)[1]][int(sheet) + 3].find('-') + 1:]
        df = pd.read_excel(pedido, sheet_name=int(sheet) - 1, header=9, usecols=['EAN', 'Cantidad'])
        df['EAN'] = df['EAN'].astype(str)
        df['Cantidad'] = df['Cantidad'].astype(str)
        df['Cantidad'] = [x.replace('.', ',') for x in df['Cantidad']]

        df2 = pd.read_excel('PROD RAPPI.xlsx', usecols=['COD_ITEM', 'EAN', 'Produto'])
        df2['COD_ITEM'] = df2['COD_ITEM'].astype(str)
        df2['EAN'] = df2['EAN'].astype(str)

        cotacao = e_text2
        df3 = pd.read_excel(cotacao, usecols=['Produto', 'Custo', 'Peso', 'Frete', 'Embalado', 'Avaria'])
        df3.replace('Em falta', np.nan, inplace=True)
        df3.replace('falta', np.nan, inplace=True)

        if datetime.today().weekday() in [4, 5, 6]:
            df3['Preço Und'] = (df3['Custo'] * 1.55 + df3['Frete'] + df3['Embalado'] + df3['Avaria']) / df3['Peso']
        else:
            df3['Preço Und'] = (df3['Custo'] * 1.45 + df3['Frete'] + df3['Embalado'] + df3['Avaria']) / df3['Peso']

        df = pd.merge(df, df2, on='EAN')

        df = pd.merge(df, df3, on='Produto')

        df['Preço Und'] = df['Preço Und'].apply(lambda x: round(x, 2))
        df['Preço Und'] = df['Preço Und'].astype(str)
        df['Preço Und'] = [x.replace('.', ',') for x in df['Preço Und']]
        df['Preço Und'] = [x.replace('nan', '0') for x in df['Preço Und']]

        gui.PAUSE = 0.2
        i = 0
        gui.keyDown('alt')
        gui.press('tab')
        gui.press('tab')
        gui.keyUp('alt')
        while i < df.shape[0]:
            qtd = df['Cantidad'][i]
            gui.write(qtd)
            gui.press('tab')
            cod = df['COD_ITEM'][i]
            gui.write(cod)
            gui.press('tab')
            gui.write(oc)
            gui.press('tab')
            preco = df['Preço Und'][i]
            gui.write(preco)
            gui.press('enter')
            i += 1

    elif cliente == 'Mundial':
        filial = 'MUND' + e_text7
        pedido = e_text6

        df = pd.read_excel(pedido, header=1)
        df.replace(np.nan, 0, inplace=True)

        df['MUND4'] = df['MUND4'].astype(int)
        df['MUND13'] = df['MUND13'].astype(int)
        df['MUND17'] = df['MUND17'].astype(int)
        df['MUND20'] = df['MUND20'].astype(int)
        df['MUND27'] = df['MUND27'].astype(int)
        df['Cod'] = df['Cod'].astype(int)

        df['MUND4'] = df['MUND4'].astype(str)
        df['MUND13'] = df['MUND13'].astype(str)
        df['MUND17'] = df['MUND17'].astype(str)
        df['MUND20'] = df['MUND20'].astype(str)
        df['MUND27'] = df['MUND27'].astype(str)
        df['Cod'] = df['Cod'].astype(str)

        df['Preço'] = df['Preço'].astype(str)
        df['Preço'] = [x.replace('.', ',') for x in df['Preço']]

        gui.PAUSE = 0.2
        i = 0
        gui.keyDown('alt')
        gui.press('tab')
        gui.press('tab')
        gui.keyUp('alt')
        while i < df.shape[0] - 1:
            qtd = df[filial][i]
            if qtd == '0':
                i += 1
                continue
            gui.write(qtd)
            gui.press('tab')
            cod = df['Cod'][i]
            gui.write(cod)
            gui.press('tab')
            gui.press('tab')
            preco = df['Preço'][i]
            gui.write(preco)
            gui.press('enter')
            i += 1

    elif cliente == 'Boa Praça':
        bp = e_text4
        pedido = e_text1
        oc = e_text5

        df = pd.read_excel(pedido, header=2, usecols=['[Referência Produto]', 'Quantidade'])
        df.replace(np.nan, 0, inplace=True)
        df['[Referência Produto]'] = df['[Referência Produto]'].astype(int)
        df['[Referência Produto]'] = df['[Referência Produto]'].astype(str)
        df['Quantidade'] = df['Quantidade'].astype(str)

        df2 = pd.read_excel('PROD BOA PRACA.xlsx', usecols=['COD_ITEM', '[Referência Produto]'])
        df2['COD_ITEM'] = df2['COD_ITEM'].astype(str)
        df2['[Referência Produto]'] = df2['[Referência Produto]'].astype(str)

        cotacao = e_text2
        df3 = pd.read_excel(cotacao)

        df3['[Referência Produto]'] = df3['[Referência Produto]'].astype(str)
        df3.replace('Em falta', np.nan, inplace=True)
        df3.replace('falta', np.nan, inplace=True)
        df3.replace('#N/D', np.nan, inplace=True)
        df3.replace('#VALOR!', np.nan, inplace=True)

        df = pd.merge(df, df2, on='[Referência Produto]')
        df = pd.merge(df, df3, how='left', on='[Referência Produto]')

        df['NOVO OFF'] = df['NOVO OFF'].apply(lambda x: round(x, 2))
        df['NOVO OFF'] = df['NOVO OFF'].astype(str)
        df['NOVO OFF'] = [x.replace('.', ',') for x in df['NOVO OFF']]
        df['NOVO OFF'] = [x.replace('nan', '0') for x in df['NOVO OFF']]

        df['NOVO LC'] = df['NOVO LC'].apply(lambda x: round(x, 2))
        df['NOVO LC'] = df['NOVO LC'].astype(str)
        df['NOVO LC'] = [x.replace('.', ',') for x in df['NOVO LC']]
        df['NOVO LC'] = [x.replace('nan', '0') for x in df['NOVO LC']]

        gui.PAUSE = 0.2
        i = 0
        gui.keyDown('alt')
        gui.press('tab')
        gui.press('tab')
        gui.keyUp('alt')
        while i < df.shape[0]:
            qtd = df['Quantidade'][i]
            gui.write(qtd)
            gui.press('tab')
            cod = df['COD_ITEM'][i]
            gui.write(cod)
            gui.press('tab')
            gui.write(oc)
            gui.press('tab')
            if bp == 'Offshore':
                preco = df['NOVO OFF'][i]
            elif bp == 'Longo curso':
                preco = df['NOVO LC'][i]
            gui.write(preco)
            gui.press('enter')
            i += 1

    elif cliente == 'Bersani':
        pedido = e_text1

        df = pd.read_excel(pedido, header = 8)
        df.drop(columns=['DT Ent.', 'Nº OC','Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], inplace=True)
        df.rename(columns={'Preço': 'Quantidade', 'Total': 'Und', 'Fornecedor': 'Preço', 'Contato / Fone': 'Total'}, inplace=True)
        df.dropna(inplace=True)
        
        df2 = pd.read_excel('PROD BERSANI.xlsx')
        
        df = pd.merge(df, df2, on='Item')
        df = df.groupby(['Item','Codigo','Produto'], as_index=False)['Quantidade'].sum()
        
        df['Quantidade'] = df['Quantidade'].apply(lambda x: round(x, 2))
        df['Quantidade'] = df['Quantidade'].astype(str)
        df['Quantidade'] = [x.replace('.', ',') for x in df['Quantidade']]
        
        df['Codigo'] = df['Codigo'].astype(str)

        cotacao = e_text2
        df3 = pd.read_excel(cotacao, usecols=['Produto', 'Custo', 'Peso', 'Frete', 'Embalado', 'Avaria'])
        df3.replace('Em falta', np.nan, inplace=True)
        df3.replace('falta', np.nan, inplace=True)

        df3['Preço Und'] = (df3['Custo'] * 1.15 + df3['Embalado'] + df3['Avaria']) / df3['Peso']

        df = pd.merge(df, df3, on='Produto')

        df['Preço Und'] = df['Preço Und'].apply(lambda x: round(x, 2))
        df['Preço Und'] = df['Preço Und'].astype(str)
        df['Preço Und'] = [x.replace('.', ',') for x in df['Preço Und']]
        df['Preço Und'] = [x.replace('nan', '0') for x in df['Preço Und']]

        gui.PAUSE = 0.2
        i = 0
        gui.keyDown('alt')
        gui.press('tab')
        gui.press('tab')
        gui.keyUp('alt')
        while i < df.shape[0]:
            qtd = df['Quantidade'][i]
            gui.write(qtd)
            gui.press('tab')
            cod = df['Codigo'][i]
            gui.write(cod)
            gui.press('tab')
            gui.press('tab')
            preco = df['Preço Und'][i]
            gui.write(preco)
            gui.press('enter')
            i += 1

In [3]:
# Cria a interface gráfica para o usuário 

janela = Tk()
janela.title("Automação de Pedidos")
janela.geometry('500x240')


def get_value():
    e_text0 = cb.get()  # cliente
    e_text1 = entry1.get()  # pedido
    e_text2 = entry2.get()  # cotação
    e_text3 = entry3.get()  # página do pedido
    e_text4 = cb_bp.get()  # tipo de navio
    e_text5 = entry4.get()  # xped
    e_text6 = entry5.get()  # mapa
    e_text7 = cb_filial.get()[:cb_filial.get().find(' -')]  # filial
    AutPed(e_text0, e_text1, e_text2, e_text3, e_text4, e_text5, e_text6, e_text7)


def verificar_opcao(event):
    # Se a opção escolhida for "Rappi", exibe o botão e a caixa de texto
    if cb.get() == "Rappi":
        texto2.grid(column=0, row=1, padx=30, pady=5, sticky="W")
        texto3.grid(column=0, row=2, padx=30, pady=5, sticky="W")
        entry1.grid(column=1, row=1, padx=5, pady=5, sticky="W")
        entry2.grid(column=1, row=2, padx=5, pady=5, sticky="W")
        botao1.grid(column=2, row=1, padx=5, pady=5, sticky="W")
        botao2.grid(column=2, row=2, padx=5, pady=5, sticky="W")
        entry3.grid(column=1, row=3, padx=5, pady=5, sticky="W")
        texto4.grid(column=0, row=3, padx=30, pady=5, sticky="W")
        texto_bp.grid_forget()
        texto_xped.grid_forget()
        cb_bp.grid_forget()
        entry4.grid_forget()
        texto_mapa.grid_forget()
        entry5.grid_forget()
        cb_filial.grid_forget()
        texto_filial.grid_forget()
        botao3.grid_forget()

    elif cb.get() == "Boa Praça":
        texto2.grid(column=0, row=1, padx=30, pady=5, sticky="W")
        texto3.grid(column=0, row=2, padx=30, pady=5, sticky="W")
        entry1.grid(column=1, row=1, padx=5, pady=5, sticky="W")
        entry2.grid(column=1, row=2, padx=5, pady=5, sticky="W")
        botao1.grid(column=2, row=1, padx=5, pady=5, sticky="W")
        botao2.grid(column=2, row=2, padx=5, pady=5, sticky="W")
        texto_bp.grid(column=0, row=4, padx=30, pady=5, sticky="W")
        texto_xped.grid(column=0, row=3, padx=30, pady=5, sticky="W")
        cb_bp.grid(column=1, row=4, padx=5, pady=5, sticky="W")
        entry4.grid(column=1, row=3, padx=5, pady=5, sticky="W")
        entry3.grid_forget()
        texto4.grid_forget()
        texto_mapa.grid_forget()
        entry5.grid_forget()
        cb_filial.grid_forget()
        texto_filial.grid_forget()
        botao3.grid_forget()

    elif cb.get() == "Mundial":
        texto_mapa.grid(column=0, row=1, padx=30, pady=5, sticky="W")
        entry5.grid(column=1, row=1, padx=5, pady=5, sticky="W")
        botao3.grid(column=2, row=1, padx=5, pady=5, sticky="W")
        cb_filial.grid(column=1, row=2, padx=5, pady=5, sticky="W")
        texto_filial.grid(column=0, row=2, padx=30, pady=5, sticky="W")
        texto_bp.grid_forget()
        texto_xped.grid_forget()
        cb_bp.grid_forget()
        entry4.grid_forget()
        entry3.grid_forget()
        texto4.grid_forget()
        entry1.grid_forget()
        entry2.grid_forget()
        texto2.grid_forget()
        texto3.grid_forget()
        botao1.grid_forget()
        botao2.grid_forget()

    elif cb.get() == "Bersani":
        entry1.grid(column=1, row=1, padx=5, pady=5, sticky="W")
        botao1.grid(column=2, row=1, padx=5, pady=5, sticky="W")
        texto2.grid(column=0, row=1, padx=30, pady=5, sticky="W")
        botao2.grid(column=2, row=2, padx=5, pady=5, sticky="W")        
        texto3.grid(column=0, row=2, padx=30, pady=5, sticky="W")
        entry2.grid(column=1, row=2, padx=5, pady=5, sticky="W")
        entry3.grid_forget()
        texto4.grid_forget()
        texto_bp.grid_forget()
        texto_xped.grid_forget()
        cb_bp.grid_forget()
        entry4.grid_forget()
        texto_mapa.grid_forget()
        entry5.grid_forget()
        botao3.grid_forget()
        cb_filial.grid_forget()
        texto_filial.grid_forget()
        botao3.grid_forget()

    else:
        entry3.grid_forget()
        texto4.grid_forget()
        texto_bp.grid_forget()
        texto_xped.grid_forget()
        cb_bp.grid_forget()
        entry4.grid_forget()
        texto_mapa.grid_forget()
        entry5.grid_forget()
        botao3.grid_forget()
        cb_filial.grid_forget()
        texto_filial.grid_forget()
        botao3.grid_forget()
        entry2.grid_forget()
        botao2.grid_forget()
        texto3.grid_forget()
        entry1.grid_forget()
        botao1.grid_forget()
        texto2.grid_forget()

def entry_focus_in(event):
    if entry1.get() == 'Digite o local do arquivo do pedido...':
        entry1.delete(0, 'end')
        entry1.config(fg="Black")


def entry_focus_out(event):
    if entry1.get() == "":
        entry1.insert(0, 'Digite o local do arquivo do pedido...')
        entry1.config(fg="gray")


def set_text(text):
    entry1.delete(0, END)
    entry1.insert(0, text)
    return


def entry_focus_in2(event):
    if entry2.get() == 'Digite o local do arquivo da cotação...':
        entry2.delete(0, 'end')
        entry2.config(fg="Black")


def entry_focus_out2(event):
    if entry2.get() == "":
        entry2.insert(0, 'Digite o local do arquivo da cotação...')
        entry2.config(fg="gray")


def set_text2(text):
    entry2.delete(0, END)
    entry2.insert(0, text)
    return


def entry_focus_in3(event):
    if entry3.get() == 'Digite a página do pedido do Rappi...':
        entry3.delete(0, 'end')
        entry3.config(fg="Black")


def entry_focus_out3(event):
    if entry3.get() == "":
        entry3.insert(0, 'Digite a página do pedido do Rappi...')
        entry3.config(fg="gray")


def set_text3(text):
    entry3.delete(0, END)
    entry3.insert(0, text)
    return


def entry_focus_in4(event):
    if entry4.get() == 'Digite o código XPED do pedido...':
        entry4.delete(0, 'end')
        entry4.config(fg="Black")


def entry_focus_out4(event):
    if entry4.get() == "":
        entry4.insert(0, 'Digite o código XPED do pedido...')
        entry4.config(fg="gray")


def set_text4(text):
    entry4.delete(0, END)
    entry4.insert(0, text)
    return


def entry_focus_in5(event):
    if entry5.get() == 'Digite o local do arquivo do mapa...':
        entry5.delete(0, 'end')
        entry5.config(fg="Black")


def entry_focus_out5(event):
    if entry5.get() == "":
        entry5.insert(0, 'Digite o local do arquivo do mapa...')
        entry5.config(fg="gray")


def set_text5(text):
    entry5.delete(0, END)
    entry5.insert(0, text)
    return


lista_clientes = ['Mundial', 'Boa Praça', 'Rappi', 'Bersani']
cb = ttk.Combobox(janela, values=lista_clientes, width=20)
cb.set('Mundial')
cb.grid(column=1, row=0, padx=5, pady=5, sticky="W")
cb.bind("<<ComboboxSelected>>", verificar_opcao)

lista_navio = ['Longo curso', 'Offshore']
cb_bp = ttk.Combobox(janela, values=lista_navio, width=20)
cb_bp.set('Longo curso')

lista_filial = ['4 - Vaz Lobo', '13 - Praça Seca', '17 - Barrinha', '20 - Freguesia', '27 - Niterói']
cb_filial = ttk.Combobox(janela, values=lista_filial, width=20)
cb_filial.set('4 - Vaz Lobo')
cb_filial.grid(column=1, row=2, padx=5, pady=5, sticky="W")

texto1 = Label(janela, text="Cliente")
texto1.grid(column=0, row=0, padx=30, pady=5, sticky="W")

texto2 = Label(janela, text="Pedido")

texto3 = Label(janela, text="Cotação")

texto4 = Label(janela, text="Pág. Pedido")

texto_bp = Label(janela, text="Tipo navio")

texto_xped = Label(janela, text="XPED")

texto_mapa = Label(janela, text="Mapa")
texto_mapa.grid(column=0, row=1, padx=30, pady=5, sticky="W")

texto_filial = Label(janela, text="Filial")
texto_filial.grid(column=0, row=2, padx=30, pady=5, sticky="W")

texto_assinatura = Label(janela, text="Feito por: Marcos P. F. Gonçalves", font=('Calibri', 8, 'italic'))
texto_assinatura.grid(column=1, row=6, padx=1, pady=5, sticky="WE")

botao1 = Button(janela, text='Selecionar',
                command=lambda: set_text(askopenfilename(title="Selecione o arquivo Excel do pedido no computador")))

entry1 = Entry(janela, width=40, font=('Calibri', 10), fg='Gray')

entry1.insert(0, 'Digite o local do arquivo do pedido...')
entry1.bind("<FocusIn>", entry_focus_in)
entry1.bind("<FocusOut>", entry_focus_out)

botao2 = Button(janela, text='Selecionar',
                command=lambda: set_text2(askopenfilename(title="Selecione o arquivo da cotação que deseja usar")))

entry2 = Entry(janela, width=40, font=('Calibri', 10), fg='Gray')
entry2.insert(0, 'Digite o local do arquivo da cotação...')
entry2.bind("<FocusIn>", entry_focus_in2)
entry2.bind("<FocusOut>", entry_focus_out2)

entry3 = Entry(janela, width=40, font=('Calibri', 10), fg='Gray')
entry3.insert(0, 'Digite a página do pedido do Rappi...')
entry3.bind("<FocusIn>", entry_focus_in3)
entry3.bind("<FocusOut>", entry_focus_out3)

entry4 = Entry(janela, width=40, font=('Calibri', 10), fg='Gray')
entry4.insert(0, 'Digite o código XPED do pedido...')
entry4.bind("<FocusIn>", entry_focus_in4)
entry4.bind("<FocusOut>", entry_focus_out4)

botao3 = Button(janela, text='Selecionar',
                command=lambda: set_text5(askopenfilename(title="Selecione o arquivo Excel do mapa do Mundial")))
botao3.grid(column=2, row=1, padx=5, pady=5, sticky="W")

entry5 = Entry(janela, width=40, font=('Calibri', 10), fg='Gray')
entry5.grid(column=1, row=1, padx=5, pady=5, sticky="W")
entry5.insert(0, 'Digite o local do arquivo do mapa...')
entry5.bind("<FocusIn>", entry_focus_in5)
entry5.bind("<FocusOut>", entry_focus_out5)

bsubmit = Button(janela, text='Confirmar', command=get_value)
bsubmit.grid(column=1, row=5, padx=5, pady=5)

janela.mainloop()